# **Record Linking and Fuzzy Matching**

In [75]:
!pip install distance
!pip install fuzzywuzzy
!pip install fuzzymatcher
!pip install recordlinkage
!pip install xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [76]:
import pandas as pd
from pathlib import Path
import fuzzymatcher
import recordlinkage

In [77]:
import warnings
warnings.filterwarnings("ignore")
import os
import numpy as np
import Levenshtein
from fuzzywuzzy import fuzz
import re
from nltk.corpus import stopwords
import distance
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from tqdm.notebook import tqdm
tqdm.pandas()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## **Example using fuzzymatcher**

For the first approach, we will try using fuzzymatcher. This package leverages sqlite’s full text search capability to try to match records in two different DataFrames.

In [8]:
#! pip install PyDrive
#!pip install --target=$nb_path recordlinkage
#!pip install --target=$nb_path fuzzymatcher

In [9]:
!pip install distance
!pip install fuzzywuzzy
!pip install fuzzymatcher
!pip install recordlinkage
!pip install xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import pandas as pd
from pathlib import Path
import fuzzymatcher
import recordlinkage

In [11]:
import warnings
warnings.filterwarnings("ignore")
import os
import numpy as np
import Levenshtein
from fuzzywuzzy import fuzz
import re
from nltk.corpus import stopwords
import distance
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from tqdm.notebook import tqdm
tqdm.pandas()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
#AllExcel = !wget "https://github.com/Hope-Power/EnitityDedup/blob/main/mappinghotelsdataset.xlsx"

In [13]:
partner1_data = pd.read_excel("https://github.com/Hope-Power/EnitityDedup/raw/main/Book1.xlsx")
partner2_data = pd.read_excel("https://github.com/Hope-Power/EnitityDedup/raw/main/Book2.xlsx")

In [14]:

#link = 'https://docs.google.com/spreadsheets/d/1jDh85maEX7pod3iuj9bmgrQ_1s2UbPhf/edit?usp=sharing&ouid=106789997326294315724&rtpof=true&sd=true'
#os.getcwd()
#os.chdir('/content/Hotelmap1')
#AllExcel = pd.ExcelFile('mappinghotelsdataset.xlsx')
#partner1_data = pd.read_excel('mappinghotelsdataset.xlsx', sheet_name="Partner1")
#partner2_data = pd.read_excel('mappinghotelsdataset.xlsx', sheet_name="Partner2")
#example_data = pd.read_excel('mappinghotelsdataset.xlsx', sheet_name="examples")


In [15]:
partner1_data.head()

,p1.key,p1.hotel_name,p1.city_name,p1.country_code,p1.hotel_address,p1.star_rating,p1.postal_code
0,5E876BFEA81A39E42E3019FE17303D52,Elite Grande Hotel,Manama,BH,"Bldg 3378, Road 2845, Area 428",4.0,5458
1,4F315989358CC0F3F7869F569887743D,Quality Inn West Chester,West Chester (OH),US,8567 Cincinnati Dayton Road,3.0,45069
2,A4EEBCBB9932DADE591248DFFFBDC068,MAP5 Village Resort,Goa,IN,Vithaldas Wadoo,3.0,403512
3,2833BE9FD49A063A36D3DE1E5E28ABC4,Hampton Inn & Suites San Jose Hotel,San Jose (CA),US,55 Old Tully Road,3.0,95111
4,F7C20B50AE5C6C807BAABB65B8926F07,Favehotel Daeng Tompo,Makassar,ID,"Daeng Tompo Street number 28, Losari",3.0,NaN


In [16]:
partner2_data.head()

,p2.key,p2.hotel_name,p2.city_name,p2.country_code,p2.hotel_address,p2.star_rating,p2.postal_code
0,9494A063EF5C6B311CDD2DDD87D775A9,MGM Luxury Villa,Uluwatu,ID,"Jl. Indraprasta, Br. Giri Darma Bukit Ungasan",2.0,80364
1,4A46417D4D1B9ED3E5B5198D7E31B7CD,The Urban Hotel,Bangalore,IN,"3, Rustam Bagh Road, Off HAL Airport Road, (ne...",3.0,560017
2,76160461BF16DEE6C10ECAACC6B576F5,Hotel Royale Ambience,Raipur,IN,"Shri Guru Gobind Singh Chowk, Fafadih",3.0,492009
3,3ACC6E5C560D54CDA969FCA1714AB3DF,Madu Inn,Jakarta,ID,Jl. Madu No. 36,1.0,11180
4,04AD88AFFF89A2B3299736E9C01B3755,Ron Gong Hotel,Chiayi City,TW,"No. 77, Ximen Street",0.0,700


In [17]:
# Columns to match on from df_left
left_on = ["p1.hotel_name","p1.city_name","p1.country_code","p1.hotel_address","p1.star_rating","p1.postal_code"]
# Columns to match on from df_right
right_on = ["p2.hotel_name","p2.city_name","p2.country_code","p2.hotel_address","p2.star_rating","p2.postal_code"]


In [18]:
# Now perform the match
# It will take several minutes to run on this data set
matched_results = fuzzymatcher.fuzzy_left_join(partner1_data,
                                               partner2_data,
                                               left_on,
                                               right_on,
                                               left_id_col='p1.key',
                                               right_id_col='p2.key')

In [19]:
matched_results.head()

,best_match_score,__id_left,__id_right,p1.key,p1.hotel_name,p1.city_name,p1.country_code,p1.hotel_address,p1.star_rating,p1.postal_code,p2.key,p2.hotel_name,p2.city_name,p2.country_code,p2.hotel_address,p2.star_rating,p2.postal_code
0,-0.398661,5E876BFEA81A39E42E3019FE17303D52,D1B48E98D68672E4EB9B8A8B06799EDE,5E876BFEA81A39E42E3019FE17303D52,Elite Grande Hotel,Manama,BH,"Bldg 3378, Road 2845, Area 428",4.0,5458,D1B48E98D68672E4EB9B8A8B06799EDE,Elite Grande,Manama,BH,Seef District,4.0,199
255,2.247304,4F315989358CC0F3F7869F569887743D,28E50B8F4185B367F6B31F9B2151D634,4F315989358CC0F3F7869F569887743D,Quality Inn West Chester,West Chester (OH),US,8567 Cincinnati Dayton Road,3.0,45069,28E50B8F4185B367F6B31F9B2151D634,Quality Inn West Chester,West Chester,US,8567 Cincinnati Dayton Road,2.0,45069
439,1.032658,A4EEBCBB9932DADE591248DFFFBDC068,1A83AC6BA04BA9A8A080888637331359,A4EEBCBB9932DADE591248DFFFBDC068,MAP5 Village Resort,Goa,IN,Vithaldas Wadoo,3.0,403512,1A83AC6BA04BA9A8A080888637331359,MAP5 Village Resort,Morjim,IN,"Vithaldas Wadoo Morjim Beach , North Goa",0.0,403512
582,2.143657,2833BE9FD49A063A36D3DE1E5E28ABC4,3F2A136272AB1BC4EE09E0695BECFFD1,2833BE9FD49A063A36D3DE1E5E28ABC4,Hampton Inn & Suites San Jose Hotel,San Jose (CA),US,55 Old Tully Road,3.0,95111,3F2A136272AB1BC4EE09E0695BECFFD1,Hampton Inn & Suites San Jose,San Jose,US,55 Old Tully Road,3.0,95111
755,1.701531,F7C20B50AE5C6C807BAABB65B8926F07,F47B75C6C93D75F5B83E721AF7561E4B,F7C20B50AE5C6C807BAABB65B8926F07,Favehotel Daeng Tompo,Makassar,ID,"Daeng Tompo Street number 28, Losari",3.0,NaN,F47B75C6C93D75F5B83E721AF7561E4B,favehotel Daeng Tompo,Makassar,ID,"Jalan Daeng Tompo No.28, Losari",3.0,90112


In [20]:
# Reorder the columns to make viewing easier
cols = [
    "best_match_score", "p1.hotel_name", "p2.hotel_name", "p1.city_name", "p1.city_name",
    "p1.country_code", "p2.country_code", "p1.hotel_address", "p2.hotel_address", "p1.postal_code", "p2.postal_code", "p1.star_rating", "p2.star_rating"
]


In [21]:
# Let's see the best matches
matched_results[cols].sort_values(by=['best_match_score'], ascending=False).head(5)

,best_match_score,p1.hotel_name,p2.hotel_name,p1.city_name,p1.city_name,p1.country_code,p2.country_code,p1.hotel_address,p2.hotel_address,p1.postal_code,p2.postal_code,p1.star_rating,p2.star_rating
546894,4.771527,Vista Rooms at VKI Industries,Vista Rooms at VKI Industries,Jaipur,Jaipur,IN,IN,"Hotel Harsh,C-162, 4-C Colony Near 14th Number...","Hotel Harsh,C-162, 4-C Colony Near 14th Number...",302013,302013,3.0,0.0
297668,4.570703,ZEN Rooms Suthisan,ZEN Rooms Suthisan,Bangkok,Bangkok,TH,TH,"Located in Double DD House, 2 Udomsuk Alley 4 ...","Located in Double DD House, 2 Udomsuk Alley 4 ...",10310,10310,3.0,1.0
466701,4.499559,Vista Rooms @ Doddabetta,Vista Rooms at Doddabetta,Ooty,Ooty,IN,IN,"Hotel RJ Inn, 5/860-3, Near 4 Road Junctions, ...","Hotel RJ Inn, 5/860-3, Near 4 Road Junctions, ...",643002,643002,3.0,0.0
748680,4.384852,Hotel Decent Guest House,Decent Guest House,Mumbai,Mumbai,IN,IN,"Ground Floor, 13-14, Plot 211, Next to Habib H...","Next Habib Hospital Near Kalpana Cinema, Lal B...",400070,400070,1.0,0.0
470363,4.351980,Al Baleed Resort Salalah By Anantara,Al Baleed Resort Salalah by Anantara,Salalah,Salalah,OM,OM,"Al Mansurah Street, P.O Box 360, P.C. 214, Al ...","P.O. Box 360, P.C. 214 Al Mansurah Street, Al ...",360,NaN,5.0,5.0


In [22]:
# Now the worst matches
matched_results[cols].sort_values(by=['best_match_score'],
                                  ascending=True).head(5)

,best_match_score,p1.hotel_name,p2.hotel_name,p1.city_name,p1.city_name,p1.country_code,p2.country_code,p1.hotel_address,p2.hotel_address,p1.postal_code,p2.postal_code,p1.star_rating,p2.star_rating
489458,-2.018128,Vista Rooms @ Phulchhab Chowk,Vista Rooms At Shastri Maidan,Rajkot,Rajkot,IN,IN,"Hotel Park Inn, Krishna Archade, Nutan Saurast...","Opposite Shashtri Maidan, Near Everest Buildin...",360001,360001,3.0,3.0
773548,-1.964686,China Sunshine Apartment Dacheng,Guangzhou Hotel,Beijing,Beijing,CN,CN,Dacheng International Centre Apartment Buildin...,No.2 Qiyi Road,NaN,510115,2.0,3.0
277766,-1.895444,7 Days Inn Dongguan Tangxia Yingfeng Shopping ...,7Days Inn Dongguan Shijie Jiarong Shopping Plaza,Dongguan,Dongguan,CN,CN,"No.120, Tangxia Avenue Middle(the opposite of ...",No.3-5 Fengge Street,523710,523290,2.0,2.0
349361,-1.802197,Anh Xuan Hotel,Lan Anh Hotel,Phan Rang - Thap Cham (Ninh Thuan),Phan Rang - Thap Cham (Ninh Thuan),VN,VN,"246 Ngo Gia Tu St, Phan Rang Thap Cham City","Yen Ninh, My Binh",NaN,NaN,1.0,0.0
827441,-1.736425,How Flowers Youth Hostel Guilin,How Flowers Youth Hostel,Guilin,Guilin,CN,CN,NO.6 Zhongshan North Road Shangzhi Lane.take t...,No.6-2 Shang Zhi Lane,541000,541002,2.0,2.0


In [23]:
# Look at the matches around 1
matched_results[cols].query("best_match_score <= 1").sort_values(
    by=['best_match_score'], ascending=False).head(10)

,best_match_score,p1.hotel_name,p2.hotel_name,p1.city_name,p1.city_name,p1.country_code,p2.country_code,p1.hotel_address,p2.hotel_address,p1.postal_code,p2.postal_code,p1.star_rating,p2.star_rating
202278,0.999997,Hotel Terminus Stockholm,Hotel Terminus Stockholm,Stockholm,Stockholm,SE,SE,Vasagatan 20,Vasagatan 20,111 20,101 25,4.0,4.0
540090,0.999597,The Nice Hotel,The Nice Krabi Hotel,Krabi,Krabi,TH,TH,"23 Chaokhun rd., Paknam",23 Chaokhun Road,81000,81000,3.0,3.0
181139,0.999217,Best Western Airport Inn,Best Western Airport Inn,Calgary (AB),Calgary (AB),CA,CA,1947 18th Avenue NE,1947 18 Avenue Northeast,T2E 7T8,T2E 7T8,2.0,3.0
330788,0.998849,Hotel Lindemann's,Lindemann'S,Berlin,Berlin,DE,DE,Potsdamer Str. 171-173.,Potsdamer Straße 171-173,NaN,10783,3.0,3.0
799460,0.998499,Legend Golf and Safari Resort,Legend Golf & Safari Resort,Naboomspruit,Naboomspruit,ZA,ZA,Haakdoring Road,Haakdoring Road,560,560,4.5,0.0
691105,0.998351,Vijittada Hostel,Vijittada Hostel,Bangkok,Bangkok,TH,TH,23 samsen road. sampraya. phranakhon.,"23 Samsen Rd., Sampraya, Phra Nakhon",10200,10200,2.0,1.0
660250,0.997906,Xiamen Plaza Hotel,Xiamen Plaza Hotel,Xiamen,Xiamen,CN,CN,908 Xiahe Road,"No.908 Xiahe Road, Siming District",361001,361004,4.0,4.0
606555,0.997589,Nova Hotel,Nova Hotel,Hanoi,Hanoi,VN,VN,75 Hang Dieu,75 Hang Dieu Street,10000,NaN,3.0,3.0
877350,0.997587,Summerside Villa-Kandy,Summerside Villa,Kandy,Kandy,LK,LK,No 35/2 Anniewatte Road,"35/2, Anniewatte Road, Anniewatte Kandy",20000,20000,1.0,0.0
37010,0.997569,Hotel Hegsagone Marine Asia,Hotel Hegsagone Marine Asia,Kocaeli,Kocaeli,TR,TR,"Balyanos Bay, Kaptan Street, No 18 Bayramoğlu ...","Balyanos Koyu, Kaptan Sk. No:18 Bayramoglu",41700,41700,4.0,4.0


In [24]:
matched_results[cols].query("best_match_score <= .10").sort_values(
    by=['best_match_score'], ascending=False).head(5)

,best_match_score,p1.hotel_name,p2.hotel_name,p1.city_name,p1.city_name,p1.country_code,p2.country_code,p1.hotel_address,p2.hotel_address,p1.postal_code,p2.postal_code,p1.star_rating,p2.star_rating
422087,0.099477,Saramo Hostel Kuching,Kuching Park Hotel,Kuching,Kuching,MY,MY,"Lot 183, 2nd Floor, Abel Road","Lot 606, Padungan Road",93150,93100,3.0,3.0
211207,0.098961,7 Days Inn Harbin West Station Haxi Shopping Mall,7Days Inn Harbin Xifuzhuang Cheng,Harbin,Harbin,CN,CN,No. 99 Zhongxing Road,No.99 Zhongxing Avenue,NaN,150086,2.0,2.0
430445,0.098788,Wingsu Guest House,Matra Bali Guesthouse,Bali,Bali,ID,ID,"Jalan Raya Pantai Berawa No. 33, Br. Pelambing...",Jl. Pantai Berawa Br Pelambingan no 37 Tibuben...,80361,80361,1.0,1.0
122291,0.098543,Thaproban Pavilion Resort and Spa,Thaproban Beach House,Unawatuna,Unawatuna,LK,LK,"Unawatuna, Galle","Yaddehimulla Road, Unawatuna",NaN,80000\t,5.0,4.0
545621,0.098185,Coco Lanta Resort,Hana Lanta Resort,Koh Lanta,Koh Lanta,TH,TH,"175 Moo. 2 Klong Khong Beach, Saladan Koh Lanta","679 Moo 2, Saladan,",81150,81150,2.5,2.0


# Note

**Overall, fuzzymatcher is a useful tool to have for medium sized data sets. As you start to get to 10,000’s of rows, it will take a lot of time to compute, so plan accordingly. However the ease of use - especially when working with pandas makes it a great first place to start.**

## **Example using Python Record Linkage Toolkit**

In [25]:
# Re-read in the data using the index_col
partner1_data = pd.read_excel("https://github.com/Hope-Power/EnitityDedup/raw/main/Book1.xlsx", index_col='p1.key')
partner2_data = pd.read_excel("https://github.com/Hope-Power/EnitityDedup/raw/main/Book2.xlsx", index_col='p2.key')


**Preprocessing**

In [26]:
STOP_WORDS = ['hotel','inn','resort','a','&','and','the','of','apartments','villa','villas','suites','house','lodge','hostel',
             'boutique','motel','spa','guesthouse','station','plaza','palace','airport']
def text_preprocessor1(df):
    cleaned = []
    for titles in tqdm(df['p1.hotel_name']):
        title = str(titles).lower()
        title = title.strip()
        title = title.replace('\\r',' ')
        title = title.replace('\\"',' ')
        title = title.replace('\\n',' ')
        title = title.replace("'s",'')
        title = re.sub('[^A-Za-z0-9]+',' ',title)
        title = re.sub('[0-9]+',' ',title)
        title = ' '.join(f for f in title.split() if f not in STOP_WORDS)
        cleaned.append(title.strip())
    return cleaned
def text_preprocessor2(df):
    cleaned = []
    for titles in tqdm(df['p2.hotel_name']):
        title = str(titles).lower()
        title = title.strip()
        title = title.replace('\\r',' ')
        title = title.replace('\\"',' ')
        title = title.replace('\\n',' ')
        title = title.replace("'s",'')
        title = re.sub('[^A-Za-z0-9]+',' ',title)
        title = re.sub('[0-9]+',' ',title)
        title = ' '.join(f for f in title.split() if f not in STOP_WORDS)
        cleaned.append(title.strip())
    return cleaned

In [27]:
cleaned_p1 = text_preprocessor1(partner1_data)
cleaned_p2 = text_preprocessor2(partner2_data)
partner1_data['p1.hotel_name'] = cleaned_p1
partner2_data['p2.hotel_name'] = cleaned_p2

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [28]:
def text_preprocessor3(df):
    cleaned = []
    for titles in tqdm(df['p1.hotel_address']):
        title = str(titles).lower()
        title = title.strip()
        title = title.replace('\\r',' ')
        title = title.replace('\\"',' ')
        title = title.replace('\\n',' ')
        title = title.replace("'s",'')
        title = re.sub('[^A-Za-z0-9]+',' ',title)
        title = re.sub('[0-9]+',' ',title)
        if pd.isnull(title):
          title = '-'
        cleaned.append(title.strip())
    return cleaned
def text_preprocessor4(df):
    cleaned = []
    for titles in tqdm(df['p2.hotel_address']):
        title = str(titles).lower()
        title = title.strip()
        title = title.replace('\\r',' ')
        title = title.replace('\\"',' ')
        title = title.replace('\\n',' ')
        title = title.replace("'s",'')
        title = re.sub('[^A-Za-z0-9]+',' ',title)
        title = re.sub('[0-9]+',' ',title)
        if pd.isnull(title):
          title = '-'
        cleaned.append(title.strip())
    return cleaned

In [29]:
cleaned_p3 = text_preprocessor3(partner1_data)
cleaned_p4 = text_preprocessor4(partner2_data)
partner1_data['p1.hotel_address'] = cleaned_p3
partner2_data['p2.hotel_address'] = cleaned_p4

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [30]:
partner1_data.head()

,p1.hotel_name,p1.city_name,p1.country_code,p1.hotel_address,p1.star_rating,p1.postal_code
p1.key,,,,,,
5E876BFEA81A39E42E3019FE17303D52,elite grande,Manama,BH,bldg road area,4.0,5458
4F315989358CC0F3F7869F569887743D,quality west chester,West Chester (OH),US,cincinnati dayton road,3.0,45069
A4EEBCBB9932DADE591248DFFFBDC068,map village,Goa,IN,vithaldas wadoo,3.0,403512
2833BE9FD49A063A36D3DE1E5E28ABC4,hampton san jose,San Jose (CA),US,old tully road,3.0,95111
F7C20B50AE5C6C807BAABB65B8926F07,favehotel daeng tompo,Makassar,ID,daeng tompo street number losari,3.0,NaN


In [31]:
partner2_data.head()

,p2.hotel_name,p2.city_name,p2.country_code,p2.hotel_address,p2.star_rating,p2.postal_code
p2.key,,,,,,
9494A063EF5C6B311CDD2DDD87D775A9,mgm luxury,Uluwatu,ID,jl indraprasta br giri darma bukit ungasan,2.0,80364
4A46417D4D1B9ED3E5B5198D7E31B7CD,urban,Bangalore,IN,rustam bagh road off hal airport road next to ...,3.0,560017
76160461BF16DEE6C10ECAACC6B576F5,royale ambience,Raipur,IN,shri guru gobind singh chowk fafadih,3.0,492009
3ACC6E5C560D54CDA969FCA1714AB3DF,madu,Jakarta,ID,jl madu no,1.0,11180
04AD88AFFF89A2B3299736E9C01B3755,ron gong,Chiayi City,TW,no ximen street,0.0,700


In [32]:
# Build the indexer
indexer = recordlinkage.Index()
# Can use full or block
#indexer.full()
#indexer.block(left_on='p1 tate', right_on='p2 State')

# Use sortedneighbor as a good option if data is not clean
indexer.sortedneighbourhood(left_on='p1.country_code', right_on='p2.country_code')

<Index>

In [33]:
candidates = indexer.index(partner1_data, partner2_data)

In [34]:
# Let's see how many matches we want to do
print(len(candidates))

5761609


In [35]:
type(partner2_data)

pandas.core.frame.DataFrame

In [36]:
type(candidates)

pandas.core.indexes.multi.MultiIndex

In [37]:
# Takes few minutes using the full index.
# few seconds using sorted neighbor
# even less using blocking
compare = recordlinkage.Compare()
compare.exact('p1.country_code', 'p2.country_code', label='country_code')
compare.exact('p1.city_name', 'p2.city_name', label='city_name')

compare.string('p1.hotel_name',
               'p2.hotel_name',
               threshold=0.85,
               label='hotel_name')
compare.string('p1.hotel_address',
               'p2.hotel_address',
               method='jarowinkler',
               threshold=0.85,
               label='hotel_address')

#compare.numeric('p1.star_rating', 'p2.star_rating')
features = compare.compute(candidates, partner1_data, partner2_data)

In [38]:
features

country_code  \
p1.key                           p2.key                                           
5E876BFEA81A39E42E3019FE17303D52 0CBA441F5B6EBC7420E3F1E691104807             0   
                                 68091C4DA3C198D5891C9960AA2B14F3             0   
                                 45F86B62D4327693BC888F6C9E00D5B0             0   
                                 87F84D10E1CD714F3774FDCDFB4A980E             0   
                                 12E270516C04CA71697D00B4340ECB1A             0   
...                                                                         ...   
B20BA40FD3E4CFEDB2CB426F7B9E89D7 F3DD4C5FC8D5D2AE163D704CBBE22927             0   
                                 86AFAFF232B86D6AE5D6C9A1F967549F             0   
                                 903FCFE68716E0D6D1EF80C011264D57             0   
                                 65C4558240C6D4A68BEE536AA6BCA958             0   
C1515D827AA010CDFDFBCD03678249B6 18E06C0BFA85265982E369E85DDC67C8             0   

                                                                   city_name  \
p1.key                           p2.key                                        
5E876BFEA81A39E42E3019FE17303D52 0CBA441F5B6EBC7420E3F1E691104807          0   
                                 68091C4DA3C198D5891C9960AA2B14F3          0   
                                 45F86B62D4327693BC888F6C9E00D5B0          0   
                                 87F84D10E1CD714F3774FDCDFB4A980E          0   
                                 12E270516C04CA71697D00B4340ECB1A          0   
...                                                                      ...   
B20BA40FD3E4CFEDB2CB426F7B9E89D7 F3DD4C5FC8D5D2AE163D704CBBE22927          0   
                                 86AFAFF232B86D6AE5D6C9A1F967549F          0   
                                 903FCFE68716E0D6D1EF80C011264D57          0   
                                 65C4558240C6D4A68BEE536AA6BCA958          0   
C1515D827AA010CDFDFBCD03678249B6 18E06C0BFA85265982E369E85DDC67C8          0   

                                                                   hotel_name  \
p1.key                           p2.key                                         
5E876BFEA81A39E42E3019FE17303D52 0CBA441F5B6EBC7420E3F1E691104807         0.0   
                                 68091C4DA3C198D5891C9960AA2B14F3         0.0   
                                 45F86B62D4327693BC888F6C9E00D5B0         0.0   
                                 87F84D10E1CD714F3774FDCDFB4A980E         0.0   
                                 12E270516C04CA71697D00B4340ECB1A         0.0   
...                                                                       ...   
B20BA40FD3E4CFEDB2CB426F7B9E89D7 F3DD4C5FC8D5D2AE163D704CBBE22927         0.0   
                                 86AFAFF232B86D6AE5D6C9A1F967549F         0.0   
                                 903FCFE68716E0D6D1EF80C011264D57         0.0   
                                 65C4558240C6D4A68BEE536AA6BCA958         0.0   
C1515D827AA010CDFDFBCD03678249B6 18E06C0BFA85265982E369E85DDC67C8         0.0   

                                                                   hotel_address  
p1.key                           p2.key                                           
5E876BFEA81A39E42E3019FE17303D52 0CBA441F5B6EBC7420E3F1E691104807            0.0  
                                 68091C4DA3C198D5891C9960AA2B14F3            0.0  
                                 45F86B62D4327693BC888F6C9E00D5B0            0.0  
                                 87F84D10E1CD714F3774FDCDFB4A980E            0.0  
                                 12E270516C04CA71697D00B4340ECB1A            0.0  
...                                                                          ...  
B20BA40FD3E4CFEDB2CB426F7B9E89D7 F3DD4C5FC8D5D2AE163D704CBBE22927            0.0  
                                 86AFAFF232B86D6AE5D6C9A1F967549F            0.0  
                                 903FCFE68716E0D6D1EF80C011264

In [39]:
# What are the score totals?
features.sum(axis=1).value_counts().sort_index(ascending=False)

4.0       2680
3.0       9226
2.0     227002
1.0    4834701
0.0     688000
dtype: int64

In [40]:
# Get the potential matches
potential_matches = features[features.sum(axis=1) > 3].reset_index()

In [41]:
potential_matches['Score'] = potential_matches.loc[:, 'country_code':'hotel_address'].sum(axis=1)
potential_matches = potential_matches.sort_values('Score', ascending=False)

In [42]:
potential_matches.head(5)

,p1.key,p2.key,country_code,city_name,hotel_name,hotel_address,Score
0,FE7B3C0B6ED8351FD9DEC136858E0A66,6C0C5E7C2B468EA041E47E8DDE3287CB,1,1,1.0,1.0,4.0
1780,A24A1387556109B3BE07295790B3FB1A,B5C1333BF462C8B2AC4A1319F2F6A1A1,1,1,1.0,1.0,4.0
1782,CFAA334557E51698B340D2C8F8A91129,9BFBE4786290D357F1D0709D05831BD8,1,1,1.0,1.0,4.0
1783,B839564C7FAAD8869BA4BEF16E5B270D,9DBC2CD22B60D23A4A6877CEBAB8F9F2,1,1,1.0,1.0,4.0
1784,8DFAD228F4D8D6959FE04F22BE909060,8E43AB90693A35961C18A0F3DD801AA6,1,1,1.0,1.0,4.0


In [43]:
partner1_data.loc["FE7B3C0B6ED8351FD9DEC136858E0A66",:]

p1.hotel_name       desert pearl apartment
p1.city_name                        Manama
p1.country_code                         BH
p1.hotel_address         road   blk   bldg
p1.star_rating                         4.0
p1.postal_code                         NaN
Name: FE7B3C0B6ED8351FD9DEC136858E0A66, dtype: object

In [44]:
partner2_data.loc["6C0C5E7C2B468EA041E47E8DDE3287CB",:]

p2.hotel_name                  desert pearl apartment
p2.city_name                                   Manama
p2.country_code                                    BH
p2.hotel_address    road   block   building   juffair
p2.star_rating                                    0.0
p2.postal_code                                    199
Name: 6C0C5E7C2B468EA041E47E8DDE3287CB, dtype: object

Yep. Those look like good matches.

Now that we know the matches, we need to wrangle the data to make it easier to review all the data together. I am going to make a concatenated name and address lookup for each of these source DataFrames.

In [45]:
# Add some convenience columns for comparing data
partner1_data['One_Name_Lookup'] = partner1_data[[
    "p1.hotel_name","p1.city_name","p1.hotel_address"
]].apply(lambda x: '_'.join(x), axis=1)

In [46]:
partner2_data['Two_Name_Lookup'] = partner2_data[[
    "p2.hotel_name","p2.city_name","p2.hotel_address"
]].apply(lambda x: '_'.join(x), axis=1)

In [47]:
Two_lookup = partner2_data[['Two_Name_Lookup']].reset_index()
One_lookup = partner1_data[['One_Name_Lookup']].reset_index()

Now merge in with the account data:

In [48]:
One_lookup.head()

,p1.key,One_Name_Lookup
0,5E876BFEA81A39E42E3019FE17303D52,elite grande_Manama_bldg road area
1,4F315989358CC0F3F7869F569887743D,quality west chester_West Chester (OH)_cincinn...
2,A4EEBCBB9932DADE591248DFFFBDC068,map village_Goa_vithaldas wadoo
3,2833BE9FD49A063A36D3DE1E5E28ABC4,hampton san jose_San Jose (CA)_old tully road
4,F7C20B50AE5C6C807BAABB65B8926F07,favehotel daeng tompo_Makassar_daeng tompo str...


In [49]:
Two_lookup.head()

,p2.key,Two_Name_Lookup
0,9494A063EF5C6B311CDD2DDD87D775A9,mgm luxury_Uluwatu_jl indraprasta br giri darm...
1,4A46417D4D1B9ED3E5B5198D7E31B7CD,urban_Bangalore_rustam bagh road off hal airpo...
2,76160461BF16DEE6C10ECAACC6B576F5,royale ambience_Raipur_shri guru gobind singh ...
3,3ACC6E5C560D54CDA969FCA1714AB3DF,madu_Jakarta_jl madu no
4,04AD88AFFF89A2B3299736E9C01B3755,ron gong_Chiayi City_no ximen street


In [50]:
One_merge = potential_matches.merge(One_lookup, how='left')

In [51]:
One_merge.head()

,p1.key,p2.key,country_code,city_name,hotel_name,hotel_address,Score,One_Name_Lookup
0,FE7B3C0B6ED8351FD9DEC136858E0A66,6C0C5E7C2B468EA041E47E8DDE3287CB,1,1,1.0,1.0,4.0,desert pearl apartment_Manama_road blk bldg
1,A24A1387556109B3BE07295790B3FB1A,B5C1333BF462C8B2AC4A1319F2F6A1A1,1,1,1.0,1.0,4.0,comfort augusta westside_Port Augusta_loudon road
2,CFAA334557E51698B340D2C8F8A91129,9BFBE4786290D357F1D0709D05831BD8,1,1,1.0,1.0,4.0,urban central accommodation_Melbourne_city road
3,B839564C7FAAD8869BA4BEF16E5B270D,9DBC2CD22B60D23A4A6877CEBAB8F9F2,1,1,1.0,1.0,4.0,flinders wharf_Melbourne_siddeley street
4,8DFAD228F4D8D6959FE04F22BE909060,8E43AB90693A35961C18A0F3DD801AA6,1,1,1.0,1.0,4.0,habitat_Broome_port drive


In [52]:
Two_lookup.head()

,p2.key,Two_Name_Lookup
0,9494A063EF5C6B311CDD2DDD87D775A9,mgm luxury_Uluwatu_jl indraprasta br giri darm...
1,4A46417D4D1B9ED3E5B5198D7E31B7CD,urban_Bangalore_rustam bagh road off hal airpo...
2,76160461BF16DEE6C10ECAACC6B576F5,royale ambience_Raipur_shri guru gobind singh ...
3,3ACC6E5C560D54CDA969FCA1714AB3DF,madu_Jakarta_jl madu no
4,04AD88AFFF89A2B3299736E9C01B3755,ron gong_Chiayi City_no ximen street


Finally, merge in the second data:

In [53]:
# Let's build a dataframe to  compare
final_merge = One_merge.merge(Two_lookup, how='left')

In [54]:
final_merge.head()

,p1.key,p2.key,country_code,city_name,hotel_name,hotel_address,Score,One_Name_Lookup,Two_Name_Lookup
0,FE7B3C0B6ED8351FD9DEC136858E0A66,6C0C5E7C2B468EA041E47E8DDE3287CB,1,1,1.0,1.0,4.0,desert pearl apartment_Manama_road blk bldg,desert pearl apartment_Manama_road block b...
1,A24A1387556109B3BE07295790B3FB1A,B5C1333BF462C8B2AC4A1319F2F6A1A1,1,1,1.0,1.0,4.0,comfort augusta westside_Port Augusta_loudon road,comfort augusta westside_Port Augusta_loudon road
2,CFAA334557E51698B340D2C8F8A91129,9BFBE4786290D357F1D0709D05831BD8,1,1,1.0,1.0,4.0,urban central accommodation_Melbourne_city road,urban central accommodation_Melbourne_city rd
3,B839564C7FAAD8869BA4BEF16E5B270D,9DBC2CD22B60D23A4A6877CEBAB8F9F2,1,1,1.0,1.0,4.0,flinders wharf_Melbourne_siddeley street,flinders wharf_Melbourne_siddeley st
4,8DFAD228F4D8D6959FE04F22BE909060,8E43AB90693A35961C18A0F3DD801AA6,1,1,1.0,1.0,4.0,habitat_Broome_port drive,habitat_Broome_port drive


Let’s see what the final data looks like:

In [55]:
cols = ['p1.key', 'p2.key', 'Score', 'One_Name_Lookup','Two_Name_Lookup']

In [56]:
final_merge[cols].sort_values(by=['Score', 'p1.key'], ascending=False)

,p1.key,p2.key,Score,One_Name_Lookup,Two_Name_Lookup
437,FFF12077AE1F7941CCBF459E25180AD7,7D62E9703BD4E22250F44140044B69D5,4.0,big heritage home_Davao City_juna avenue matina,big heritage home_Davao City_juna avenue matina
1218,FFCE3D746B512B0F5AEB18B642380C98,404B3720EBE941DEAC24A8F1C6FBB97C,4.0,wombats city budapest_Budapest_kiraly street,wombats city budapest_Budapest_kiraly u
710,FFC1382422B020D20B101BBF69B6F3DB,2E4C0FFFC8488719169AE93B5C17B66E,4.0,best western museum_Athens_bouboulinas street,best western museum_Athens_bouboulinas street
248,FFA413D1C555A8D65A9CEACB858937C1,31BCEFD961447C2D497710A2878AC532,4.0,madushanka_Ella_wamullahena,madushanka_Ella_wamullahena
2521,FF8A69427D47D12D1BA30B53B9882913,2743720856EEB11422B9B7ABF6D56A2B,4.0,sathorn vista bangkok marriott executive_Bangk...,sathorn vista bangkok marriott executive_Bangk...
...,...,...,...,...,...
1700,00676BB1706C67C832A30EFC0946C61B,EA270E211270791E3C733CFDB8F1C266,4.0,gazala_Palakkad_near head post office palakkad,gazala_Palakkad_near head post office sulthanpet
1143,004DFA0326E18929831E75DEBF9741FD,41F1105D49BA928F50171BCCBEBCCD0F,4.0,ariki backpackers_New Plymouth_ariki street,ariki backpackers_New Plymouth_ariki street
753,0035AE72E4BF57671FC69B8CF98D9E09,7FF9ECD5D9BCA1EC7E740A92C6C2B8B3,4.0,tschugge_Zermatt_bodmenstrasse,tschugge_Zermatt_bodmenstrasse
2591,0034BE868C76BA8C73293D980C21884B,FE2AD7D9A9821598EB97DBDA95FE202C,4.0,na nirand romantic_Chiang Mai_soi chalernpra...,na nirand romantic_Chiang Mai_soi charoenpra...


One of the differences between the toolkit approach and fuzzymatcher is that we are including multiple matches. For instance, p1.key  "FFFF7983450A8A142DD650E09E73B4CA" could match two providers:

In [57]:
final_merge[cols].sort_values(by=['p1.key','Score'], ascending=False)

,p1.key,p2.key,Score,One_Name_Lookup,Two_Name_Lookup
437,FFF12077AE1F7941CCBF459E25180AD7,7D62E9703BD4E22250F44140044B69D5,4.0,big heritage home_Davao City_juna avenue matina,big heritage home_Davao City_juna avenue matina
1218,FFCE3D746B512B0F5AEB18B642380C98,404B3720EBE941DEAC24A8F1C6FBB97C,4.0,wombats city budapest_Budapest_kiraly street,wombats city budapest_Budapest_kiraly u
710,FFC1382422B020D20B101BBF69B6F3DB,2E4C0FFFC8488719169AE93B5C17B66E,4.0,best western museum_Athens_bouboulinas street,best western museum_Athens_bouboulinas street
248,FFA413D1C555A8D65A9CEACB858937C1,31BCEFD961447C2D497710A2878AC532,4.0,madushanka_Ella_wamullahena,madushanka_Ella_wamullahena
2521,FF8A69427D47D12D1BA30B53B9882913,2743720856EEB11422B9B7ABF6D56A2B,4.0,sathorn vista bangkok marriott executive_Bangk...,sathorn vista bangkok marriott executive_Bangk...
...,...,...,...,...,...
1700,00676BB1706C67C832A30EFC0946C61B,EA270E211270791E3C733CFDB8F1C266,4.0,gazala_Palakkad_near head post office palakkad,gazala_Palakkad_near head post office sulthanpet
1143,004DFA0326E18929831E75DEBF9741FD,41F1105D49BA928F50171BCCBEBCCD0F,4.0,ariki backpackers_New Plymouth_ariki street,ariki backpackers_New Plymouth_ariki street
753,0035AE72E4BF57671FC69B8CF98D9E09,7FF9ECD5D9BCA1EC7E740A92C6C2B8B3,4.0,tschugge_Zermatt_bodmenstrasse,tschugge_Zermatt_bodmenstrasse
2591,0034BE868C76BA8C73293D980C21884B,FE2AD7D9A9821598EB97DBDA95FE202C,4.0,na nirand romantic_Chiang Mai_soi chalernpra...,na nirand romantic_Chiang Mai_soi charoenpra...


In [58]:
final_merge[final_merge['p1.key']=="FFFF7983450A8A142DD650E09E73B4CA"][cols]

,p1.key,p2.key,Score,One_Name_Lookup,Two_Name_Lookup


In [59]:
final_merge[final_merge['p1.key']=="FFE5ED550FDD4BD449B78E5ADCA208F2"][cols]

,p1.key,p2.key,Score,One_Name_Lookup,Two_Name_Lookup


In [60]:
# If you need to save it to Excel -
#final_merge.sort_values(by=['Account_Num', 'Score'],
#                        ascending=False).to_excel('merge_list.xlsx',
#                                                  index=False)

# `Note

**As you can see from this example, the Record Linkage Toolkit allows a lot more flexibility and customization than fuzzymatcher. The downside is that there is a little more manipulation to get the data stitched back together in order to hand the data over to a person to complete the comparison.**



# **Dedupe the data**

One of the additional uses of the Record Linkage Toolkit is for finding duplicate records in a data set. The process is very similar to matching except you pass match a single DataFrame against itself.

Let’s walk through an example using a similar data set:

In [61]:
partner1_dupes = partner1_data.copy()

In [62]:
partner1_dupes.head()

,p1.hotel_name,p1.city_name,p1.country_code,p1.hotel_address,p1.star_rating,p1.postal_code,One_Name_Lookup
p1.key,,,,,,,
5E876BFEA81A39E42E3019FE17303D52,elite grande,Manama,BH,bldg road area,4.0,5458,elite grande_Manama_bldg road area
4F315989358CC0F3F7869F569887743D,quality west chester,West Chester (OH),US,cincinnati dayton road,3.0,45069,quality west chester_West Chester (OH)_cincinn...
A4EEBCBB9932DADE591248DFFFBDC068,map village,Goa,IN,vithaldas wadoo,3.0,403512,map village_Goa_vithaldas wadoo
2833BE9FD49A063A36D3DE1E5E28ABC4,hampton san jose,San Jose (CA),US,old tully road,3.0,95111,hampton san jose_San Jose (CA)_old tully road
F7C20B50AE5C6C807BAABB65B8926F07,favehotel daeng tompo,Makassar,ID,daeng tompo street number losari,3.0,NaN,favehotel daeng tompo_Makassar_daeng tompo str...


In [63]:
partner1_dupes.drop(columns=partner1_dupes.columns[-1], 
        axis=1, 
        inplace=True)

In [64]:
partner1_dupes.head()

,p1.hotel_name,p1.city_name,p1.country_code,p1.hotel_address,p1.star_rating,p1.postal_code
p1.key,,,,,,
5E876BFEA81A39E42E3019FE17303D52,elite grande,Manama,BH,bldg road area,4.0,5458
4F315989358CC0F3F7869F569887743D,quality west chester,West Chester (OH),US,cincinnati dayton road,3.0,45069
A4EEBCBB9932DADE591248DFFFBDC068,map village,Goa,IN,vithaldas wadoo,3.0,403512
2833BE9FD49A063A36D3DE1E5E28ABC4,hampton san jose,San Jose (CA),US,old tully road,3.0,95111
F7C20B50AE5C6C807BAABB65B8926F07,favehotel daeng tompo,Makassar,ID,daeng tompo street number losari,3.0,NaN


In [65]:
partner1_dupes['p1.postal_code'] = partner1_dupes['p1.postal_code'].fillna(0)
partner1_dupes['p1.postal_code'] = partner1_dupes.to_string(columns = ['p1.postal_code'])
#partner1_dupes["p1."] = partner1_dupes[''].astype(str) +"-"+ partner1_dupes["p1.postal_code"]

Then create our indexer with a sorted neighbor block on city.

In [66]:
# Deduping follows the same process, you just use 1 single dataframe
dupe_indexer = recordlinkage.Index()
dupe_indexer.sortedneighbourhood(left_on='p1.city_name')
dupe_candidate_links = dupe_indexer.index(partner1_dupes)

We should check for duplicates based on city, name and address:

In [67]:
# Comparison step
compare_dupes = recordlinkage.Compare()

compare_dupes.exact('p1.country_code', 'p1.country_code', label='country_code')
compare_dupes.exact('p1.city_name', 'p1.city_name', label='city_name')

compare_dupes.string('p1.hotel_name',
               'p1.hotel_name',
               threshold=0.85,
               label='hotel_name')
#compare_dupes.string('p1.postal_code',
#               'p1.postal_code',
#               threshold=0.85,
#               label='postal_code')
compare_dupes.string('p1.hotel_address',
               'p1.hotel_address',
               method='jarowinkler',
               threshold=0.85,
               label='hotel_address')

dupe_features = compare_dupes.compute(dupe_candidate_links, partner1_dupes)


Because we are only comparing with a single DataFrame, the resulting DataFrame has a p1.key_1 and p1.key_2:

In [68]:
dupe_features

country_code  \
p1.key_1                         p1.key_2                                         
D88A999F9BB5D6D2FD1F767DDE9B9681 6CAB10CF7994593316386D20717E24BA             0   
9DB8BBA275801554BC4CF8B731E4FD17 6CAB10CF7994593316386D20717E24BA             0   
DC4BD668E8AAB36D092C7FEF4CCAEF0E 6CAB10CF7994593316386D20717E24BA             0   
9898C631F2D747E30EDDB0740C42C003 E6110F8AF732EE83933ED1A582617A12             1   
3C79FB5061E89AE291F5A48D873406A5 E6110F8AF732EE83933ED1A582617A12             1   
...                                                                         ...   
0E1E85A01CEFEBD4D40EC9B7E2CDF200 9621BF642A356F5698546A5902246D4F             0   
D46E4722649FEA1A840B8FF16A68C697 590268AA892DBC0123C8B6E8B3746B39             1   
                                 830B78595666FF6D19A75A5FDCEC990F             1   
B7713C001E9B2CA7E28E9F1AF61AFFE2 570302EA86F116EEDC717F58A6F3573F             0   
                                 DD02907DAF43FA13D01A41E7FED3566C             0   

                                                                   city_name  \
p1.key_1                         p1.key_2                                      
D88A999F9BB5D6D2FD1F767DDE9B9681 6CAB10CF7994593316386D20717E24BA          0   
9DB8BBA275801554BC4CF8B731E4FD17 6CAB10CF7994593316386D20717E24BA          0   
DC4BD668E8AAB36D092C7FEF4CCAEF0E 6CAB10CF7994593316386D20717E24BA          0   
9898C631F2D747E30EDDB0740C42C003 E6110F8AF732EE83933ED1A582617A12          0   
3C79FB5061E89AE291F5A48D873406A5 E6110F8AF732EE83933ED1A582617A12          0   
...                                                                      ...   
0E1E85A01CEFEBD4D40EC9B7E2CDF200 9621BF642A356F5698546A5902246D4F          0   
D46E4722649FEA1A840B8FF16A68C697 590268AA892DBC0123C8B6E8B3746B39          0   
                                 830B78595666FF6D19A75A5FDCEC990F          0   
B7713C001E9B2CA7E28E9F1AF61AFFE2 570302EA86F116EEDC717F58A6F3573F          0   
                                 DD02907DAF43FA13D01A41E7FED3566C          0   

                                                                   hotel_name  \
p1.key_1                         p1.key_2                                       
D88A999F9BB5D6D2FD1F767DDE9B9681 6CAB10CF7994593316386D20717E24BA         0.0   
9DB8BBA275801554BC4CF8B731E4FD17 6CAB10CF7994593316386D20717E24BA         0.0   
DC4BD668E8AAB36D092C7FEF4CCAEF0E 6CAB10CF7994593316386D20717E24BA         0.0   
9898C631F2D747E30EDDB0740C42C003 E6110F8AF732EE83933ED1A582617A12         0.0   
3C79FB5061E89AE291F5A48D873406A5 E6110F8AF732EE83933ED1A582617A12         0.0   
...                                                                       ...   
0E1E85A01CEFEBD4D40EC9B7E2CDF200 9621BF642A356F5698546A5902246D4F         0.0   
D46E4722649FEA1A840B8FF16A68C697 590268AA892DBC0123C8B6E8B3746B39         0.0   
                                 830B78595666FF6D19A75A5FDCEC990F         0.0   
B7713C001E9B2CA7E28E9F1AF61AFFE2 570302EA86F116EEDC717F58A6F3573F         0.0   
                                 DD02907DAF43FA13D01A41E7FED3566C         0.0   

                                                                   hotel_address  
p1.key_1                         p1.key_2                                         
D88A999F9BB5D6D2FD1F767DDE9B9681 6CAB10CF7994593316386D20717E24BA            0.0  
9DB8BBA275801554BC4CF8B731E4FD17 6CAB10CF7994593316386D20717E24BA            0.0  
DC4BD668E8AAB36D092C7FEF4CCAEF0E 6CAB10CF7994593316386D20717E24BA            0.0  
9898C631F2D747E30EDDB0740C42C003 E6110F8AF732EE83933ED1A582617A12            0.0  
3C79FB5061E89AE291F5A48D873406A5 E6110F8AF732EE83933ED1A582617A12            0.0  
...                                                                          ...  
0E1E85A01CEFEBD4D40EC9B7E2CDF200 9621BF642A356F5698546A5902246D4F            0.0  
D46E4722649FEA1A840B8FF16A68C697 590268AA892DBC0123C8B6E8B3746B39            0.0  
                                 830B78595666FF6D19A75A5FDCEC9

Here is how we score:


In [69]:
dupe_features.sum(axis=1).value_counts().sort_index(ascending=False)

4.0         3
3.0      5238
2.0    172266
1.0      8032
0.0     38873
dtype: int64

Add the score column:

In [70]:
potential_dupes = dupe_features[dupe_features.sum(axis=1) > 2].reset_index()
potential_dupes['Score'] = potential_dupes.loc[:, 'country_code':'hotel_address'].sum(axis=1)

Here’s a sample:

In [71]:
potential_dupessort = potential_dupes.sort_values(by=['Score'], ascending=True)
potential_dupessort.head()

,p1.key_1,p1.key_2,country_code,city_name,hotel_name,hotel_address,Score
0,51966EDF4C6526C11EC501F72C171C4B,FE7B3C0B6ED8351FD9DEC136858E0A66,1,1,0.0,1.0,3.0
3497,9B4BE5FB0A9670A2A6E55FFA66DA0188,7592ACE11F295DBE281A2A908B314F12,1,1,0.0,1.0,3.0
3496,9B4BE5FB0A9670A2A6E55FFA66DA0188,1FCDAEEB2F3492B18AA16CD1E6217F09,1,1,0.0,1.0,3.0
3495,6A52DB09E45A58B3E50BCC6213785282,EA98CB4EF75B56ACDA616E5166438FB6,1,1,0.0,1.0,3.0
3494,6A52DB09E45A58B3E50BCC6213785282,1FCF31AAFBFCE2375F45F315A5628117,1,1,0.0,1.0,3.0


These records have a high likelihood of being duplicated. Let’s look at an example to see if they might be dupes:

In [72]:
partner1_dupes.loc["51966EDF4C6526C11EC501F72C171C4B", :]

p1.hotel_name                                 infinity infinity tower
p1.city_name                                                   Manama
p1.country_code                                                    BH
p1.hotel_address               road   bld   block   cid street adliya
p1.star_rating                                                    3.0
p1.postal_code                                              p1.pos...
Name: 51966EDF4C6526C11EC501F72C171C4B, dtype: object

In [73]:
partner1_dupes.loc["FE7B3C0B6ED8351FD9DEC136858E0A66", :]

p1.hotel_name                                  desert pearl apartment
p1.city_name                                                   Manama
p1.country_code                                                    BH
p1.hotel_address                                    road   blk   bldg
p1.star_rating                                                    4.0
p1.postal_code                                              p1.pos...
Name: FE7B3C0B6ED8351FD9DEC136858E0A66, dtype: object

In [74]:
# Take a look at one of the potential duplicates
partner1_dupes[partner1_dupes.index.isin(["9B4BE5FB0A9670A2A6E55FFA66DA0188", "7592ACE11F295DBE281A2A908B314F12"])]

,p1.hotel_name,p1.city_name,p1.country_code,p1.hotel_address,p1.star_rating,p1.postal_code
p1.key,,,,,,
7592ACE11F295DBE281A2A908B314F12,park prestige paris grande bibliotheque,Paris,FR,avenue de france,4.0,p1.pos...
9B4BE5FB0A9670A2A6E55FFA66DA0188,amhotel italie,Paris,FR,avenue de choisy,2.0,p1.pos...


Summary

Linking different record sets on text fields like names and addresses is a common but challenging data problem. The python ecosystem contains two useful libraries that can take data sets and use multiple algorithms to try to match them together.

Fuzzymatcher uses sqlite’s full text search to simply match two pandas DataFrames together using probabilistic record linkage. If you have a larger data set or need to use more complex matching logic, then the Python Record Linkage Toolkit is a very powerful set of tools for joining data and removing duplicates.

Part of my motivation for writing this long article is that there are lots of commercial options out there for these problems and I wanted to raise awareness about these python options. Before you engage with an expensive consultant or try to pay for solution, you should spend an afternoon with these two options and see if it helps you out. All of the relevant code examples to get you started are in this notebook.

I always like to hear if you find these topics useful and applicable to your own needs. Feel free to comment below and let me know if you use these or any other similar tools.